# Agentic Advanced RAG for Question Generation

This notebook implements an agentic system for generating questions from a document. It leverages advanced RAG techniques and an autonomous agent to automate and refine the question generation process.

The agent's workflow consists of four key features:
1.  **Automated Query Generation**: Automatically creates initial questions/topics from the document.
2.  **Iterative Refinement**: Employs a generate-critique-improve loop to enhance question quality.
3.  **Self-Correction/Validation**: Checks generated questions against the source document to ensure they are answerable and accurate.
4.  **Dynamic Tool Use**: Can independently decide to use external tools, like a web search, to gather more context and enrich the questions.

## 1. Setup and Dependencies

First, we install and import the necessary libraries. We'll need libraries for document loading, splitting, embeddings, vector storage, and the LangChain framework to build our agent.

In [ ]:
!pip install -qU langchain langchain-community langchain-huggingface torch transformers faiss-cpu sentence-transformers beautifulsoup4 duckduckgo-search
!pip install -qU pypdf

In [ ]:
import os
import requests
from getpass import getpass

# LangChain components
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceInstructEmbeddings, HuggingFaceEndpoint
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent, Tool
from langchain_community.tools import DuckDuckGoSearchRun

## 2. Configure Core RAG Pipeline

Here, we set up the core components of our advanced RAG system, based on the techniques in your `rag.ipynb` notebook. This includes setting up the embedding model, the LLM endpoint, and loading the source document.

In [ ]:
# Securely get Hugging Face API token
if not (hf_token := os.environ.get("HUGGINGFACEHUB_API_TOKEN")):
    hf_token = getpass("Enter your Hugging Face API token: ")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token

# Download the source PDF document (e.g., "Attention Is All You Need")
pdf_url = "https://arxiv.org/pdf/1706.03762.pdf"
pdf_path = "attention_is_all_you_need.pdf"
if not os.path.exists(pdf_path):
    response = requests.get(pdf_url)
    with open(pdf_path, "wb") as f:
        f.write(response.content)

print(f"Document downloaded to {pdf_path}")

In [ ]:
# Load and process the document
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(documents)

print(f"Document split into {len(docs)} chunks.")

In [ ]:
# Set up the embedding model (Instructor-XL)
embedding_model = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-xl",
    model_kwargs={"device": "cpu"} # Use 'cuda' if GPU is available
)

# Create the FAISS vector store
vector_db = FAISS.from_documents(docs, embedding_model)

# Create the retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 5})

print("Vector store and retriever created successfully.")

In [ ]:
# Set up the LLM (Flan-T5-XXL)
llm = HuggingFaceEndpoint(
    repo_id="google/flan-t5-xxl",
    temperature=0.7,
    max_new_tokens=300,
    top_k=30,
    huggingfacehub_api_token=hf_token
)

# Create the base RetrievalQA chain for answering questions
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

print("LLM and QA chain are ready.")

## 3. Implement the Agentic Workflow

Now, we build the agent that automates the question generation process. This involves:
1.  **Defining Tools**: A tool for answering questions using our RAG pipeline and a web search tool.
2.  **Creating the Agent**: Designing a prompt that instructs the agent to follow the generate-critique-refine loop.
3.  **Running the Agent Executor**: Kicking off the autonomous process.

In [ ]:
# 1. Define Tools for the Agent

# Tool to use the RAG pipeline for answering/validation
rag_tool = Tool(
    name="document_qa_system",
    func=lambda q: qa_chain({"query": q}),
    description="Use this to answer questions or validate information against the source document. Input is the question to be answered."
)

# Tool for web search
search_tool = DuckDuckGoSearchRun()

tools = [rag_tool, search_tool]

In [ ]:
# 2. Create the Agent

# The core prompt that drives the agent's behavior
agent_prompt_template = PromptTemplate.from_template("""
You are an expert agent designed to generate high-quality, insightful questions from a source document.
Your goal is to autonomously explore a topic within the document and formulate a question that is clear, relevant, and answerable by the document's content.

You must follow this iterative process:
1.  **Generate**: Start with an initial question based on the given topic.
2.  **Critique**: Analyze the question. Is it clear? Is it specific? Can it likely be answered by the `document_qa_system`? Does it require external knowledge? If the document context seems insufficient for a deep question, decide if a web search could provide useful context (e.g., for definitions or related concepts).
3.  **Refine**: If the critique reveals weaknesses OR if you used the web search to find more context, rewrite the question to improve it. A good refined question is often more specific and nuanced.
4.  **Validate**: Use the `document_qa_system` with your refined question. If the system provides a confident, well-supported answer, the question is valid. If the answer is poor or irrelevant, you may need to refine the question again.

TOOLS:
------
You have access to the following tools: {tools}

To use a tool, please use the following format:
```
Thought: Do I need to use a tool? Yes
Action: The action to take, should be one of [{tool_names}]
Action Input: The input to the action
Observation: The result of the action
```

When you have a final, validated question, you MUST output it in the following format:
```
Thought: I have a final, validated question.
Final Answer: [Your final, high-quality question here]
```

Begin!

Topic to explore: {input}
Thought Process Log: {agent_scratchpad}
""")

agent = create_react_agent(llm, tools, agent_prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

## 4. Run the Agent and Generate Questions

Now, we'll kickstart the process. First, we'll do a simple LLM call to generate a few high-level topics from the document. Then, we'll feed these topics to our agent one by one to generate refined, validated questions.

In [ ]:
# Step 1: Automatically generate initial topics from the document
topic_generation_prompt = PromptTemplate.from_template(
    "Based on the first few pages of the provided document, list 3-5 high-level topics or concepts that could be used to generate detailed questions. \
     Focus on the core ideas presented. Return the topics as a numbered list.\
     Document context: {context}"
)

initial_context = "\n\n".join([doc.page_content for doc in docs[:3]]) # Use first 3 chunks

topic_generation_chain = topic_generation_prompt | llm
generated_topics_str = topic_generation_chain.invoke({"context": initial_context})

topics = [line.strip() for line in generated_topics_str.split('
') if line.strip() and line[0].isdigit()]

print("Automatically Generated Topics:")
for topic in topics:
    print(topic)

In [ ]:
# Step 2: Run the agent on each topic to generate a refined question
final_questions = []
for topic in topics:
    print(f"--- Running Agent for Topic: '{topic}' ---")
    try:
        result = agent_executor.invoke({"input": topic})
        final_questions.append(result['output'])
    except Exception as e:
        print(f"An error occurred while processing topic '{topic}': {e}")
        final_questions.append(f"Failed to generate question for topic: {topic}")
    print("---")

print("\n=== Final Generated Questions ===")
for i, q in enumerate(final_questions):
    print(f"{i+1}. {q}")